In [1]:
!pip install kagglehub==0.3.3

  Using cached kagglehub-0.3.3-py3-none-any.whl.metadata (22 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached kagglehub-0.3.3-py3-none-any.whl (42 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [51]:
import kagglehub
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
# Download latest version of kaggle dataset
path = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")

In [20]:
# Get image paths and create lists to populate later
class_path = path + '/images' + '/images'
classes = os.listdir(class_path)
image_paths = []
labels = [] # Trash category associated with each image
d_r = [] # default (studio/standard) vs real world img


In [6]:
# Access image folders and populate lists with needed data
'''Arthur's shit
for i, label in enumerate(classes):
    class_dir = os.path.join(class_path, label)
    for subfolder in ['default', 'real_world']:
        subfolder_dir = os.path.join(class_dir, subfolder)
        image_names = os.listdir(subfolder_dir)
        
        for j in range(0, len(image_names), int(len(image_names)/3)): # TODO: change step on turn-in
          image_name = image_names[j]
          d_r.append(subfolder)
          image_paths.append(os.path.join(subfolder_dir, image_name)) 
          labels.append(label)
'''

In [21]:
# Access image folders and populate lists with needed data
# Serena's shit
for i, label in enumerate(classes):
    class_dir = os.path.join(class_path, label)
    for subfolder in ['default', 'real_world']:
        subfolder_dir = os.path.join(class_dir, subfolder)
        image_names = os.listdir(subfolder_dir)
        
        for image_name in image_names:
            d_r.append(subfolder)
            image_paths.append(os.path.join(subfolder_dir, image_name))
            labels.append(label)

In [25]:
# Load all data into df
columnDict = {'image_path': image_paths, 'label': labels, 'default_vs_real_world': d_r}
df = pd.DataFrame(columnDict)
df.shape

(15000, 3)

## Clean Data

In [31]:
clean_df = df.copy()
clean_df.label.unique()
cat_dic = {
    'landfill' : ['clothing', 'disposable_plastic_cutlery', 'plastic_shopping_bags', 'plastic_trash_bags',\
                  'shoes', 'styrofoam_cups', 'styrofoam_food_containers', 'paper_cups', 'plastic_straws', 'plastic_cup_lids'],
    'recyclable' : ['aerosol_cans','aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'glass_beverage_bottles',\
                    'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'plastic_detergent_bottles',\
                    'plastic_food_containers','plastic_soda_bottles', 'plastic_water_bottles', 'steel_food_cans'],
    'compost' : ['coffee_grounds', 'eggshells', 'food_waste', 'tea_bags']
}

# Create a reverse lookup dictionary to map items to categories
item_to_category = {item: category for category, items in cat_dic.items() for item in items}

# Map the labels to respective categories using lookup dictionary
clean_df['category'] = df['label'].map(item_to_category)
clean_df.head()

,image_path,label,default_vs_real_world,category
0,C:\Users\S\.cache\kagglehub\datasets\alistairk...,aerosol_cans,default,recyclable
1,C:\Users\S\.cache\kagglehub\datasets\alistairk...,aerosol_cans,default,recyclable
2,C:\Users\S\.cache\kagglehub\datasets\alistairk...,aerosol_cans,default,recyclable
3,C:\Users\S\.cache\kagglehub\datasets\alistairk...,aerosol_cans,default,recyclable
4,C:\Users\S\.cache\kagglehub\datasets\alistairk...,aerosol_cans,default,recyclable


In [35]:
def subset(df, num):
    """
    Gets subset of dataframe. USE THIS FOR WORKING LOCALLY ONLY.

    ARGS:
        df: dataframe 
        num(float): decimal representing what amt of original df you wanna use for subset (eg. .5 or .3)
    RETURNS:
        df: smaller df that's shuffled
    """
    return df.sample(frac=num, random_state=42).reset_index(drop=True)
    
# CHANGE frac TO 1 WHEN USING SUPER COMPUTER 
frac = 0.25 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
tiny_df = subset(clean_df, frac)
tiny_df.shape

(3750, 4)

In [58]:
def train_val_test(df):
    """
    Split df into train, val, test. HARDCODED to 60% train, 20% val and 20% test btw
    """
    # Get rid of unnecessary columns HARDCODED
    df = df[['image_path', 'category']]
    # Split into train and temp (40% for validation + test)
    train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
    
    # Split temp into validation and test (10% each)
    val_df, test_df = train_test_split(df, test_size=0.5, random_state=42)
    
    print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")
    return train_df, val_df, test_df

train_df, val_df, test_df = train_val_test(tiny_df)
train.head()

Train size: 2250, Validation size: 1875, Test size: 1875


,image_path,category
1190,C:\Users\S\.cache\kagglehub\datasets\alistairk...,recyclable
3411,C:\Users\S\.cache\kagglehub\datasets\alistairk...,landfill
530,C:\Users\S\.cache\kagglehub\datasets\alistairk...,recyclable
2501,C:\Users\S\.cache\kagglehub\datasets\alistairk...,recyclable
49,C:\Users\S\.cache\kagglehub\datasets\alistairk...,landfill


In [60]:
def to_tensorflow(df):
    """
    Convert pandas df to tensorflow dataset.
    ARGS:
        df: pandas df
    RETURNS 
        gen: tensorflow dataset
    """
    preprocess = tf.keras.applications.vgg16.preprocess_input # preprocessing function for CNN 
    target_size=(224,224) # set the size of the images
    color_mode='rgb' # set the type of image
    class_mode= 'categorical' # set the class mode
    batch_size=10  # set the batch size
    shuffle=False # don't shuffle cuz we alrdy shuffled our df
    gen=ImageDataGenerator(preprocessing_function=preprocess).flow_from_dataframe(df, 
          x_col='image_path',
          y_col='category', target_size=target_size, color_mode=color_mode,
          class_mode=class_mode, batch_size=batch_size, shuffle=shuffle)
    return gen
pandas_dfs = [train_df, val_df, test_df]
train_batches, val_batches, test_batches = [to_tensorflow(df) for df in pandas_dfs]

Found 2250 validated image filenames belonging to 3 classes.
Found 1875 validated image filenames belonging to 3 classes.
Found 1875 validated image filenames belonging to 3 classes.


In [42]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  0


In [57]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

1